Setup

In [1]:
import pandas as pd

# Set data files paths
FLOWS_PATH = "./data/DOS/15-02/20-03-2024-18h/netflows.csv"
CPU_PATH = "./data/DOS/15-02/20-03-2024-18h/CPU Usage-data-as-joinbyfield-2024-03-21 03_14_41.csv"
save_directory = "./output/DOS/"
start_date = '2024-03-20 21:54:40'
end_date = '2024-03-21 06:56:17'

df_flow = pd.read_csv(FLOWS_PATH)
print(df_flow.shape)
df_flow.head()
print(df_flow.nunique())

(49145, 80)
class                   2
dst_port             6658
protocol                2
timestamp           11413
flow_duration       27814
                    ...  
cwe_flag_count          1
subflow_fwd_pkts      231
subflow_bwd_pkts      234
subflow_fwd_byts     4477
subflow_bwd_byts     3492
Length: 80, dtype: int64


Time filtering

In [20]:
df_flow['timestamp'] = pd.to_datetime(df_flow['timestamp'])

print(df_flow.shape)

# Filter rows between start_date and end_date
df_flow = df_flow[(df_flow['timestamp'] >= start_date) & (df_flow['timestamp'] <= end_date)]

print(df_flow.shape)

# Replace class values ([0] -> 0 and [1] -> 1)
df_flow['class'] = df_flow['class'].replace({'[0]': 0, '[1]': 1})
# filtered_df.head()
df_flow[df_flow['class']==1]

(49145, 80)
(48839, 80)


C:\Users\hr_mo\AppData\Local\Temp\ipykernel_11972\2052669154.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_flow['class'] = df_flow['class'].replace({'[0]': 0, '[1]': 1})


,class,dst_port,protocol,timestamp,flow_duration,flow_byts_s,flow_pkts_s,fwd_pkts_s,bwd_pkts_s,tot_fwd_pkts,...,bwd_pkts_b_avg,fwd_blk_rate_avg,bwd_blk_rate_avg,fwd_seg_size_avg,bwd_seg_size_avg,cwe_flag_count,subflow_fwd_pkts,subflow_bwd_pkts,subflow_fwd_byts,subflow_bwd_byts
865,1,80,6,2024-03-20 22:17:52,5.303789e+06,236.623305,1.885445,1.131267,0.754178,6,...,0.0,0.000000,0.0,90.166667,178.500000,0,6,4,541,714
1207,1,22,6,2024-03-20 22:29:16,6.632161e+04,8458.781205,120.624331,75.390207,45.234124,5,...,0.0,0.000000,0.0,62.800000,82.333333,0,5,3,314,247
2039,1,80,6,2024-03-20 22:57:14,4.300155e+06,422.542892,1.860398,1.162749,0.697649,5,...,0.0,0.000000,0.0,127.800000,392.666667,0,5,3,639,1178
2046,1,80,6,2024-03-20 22:57:14,3.528595e+02,583801.772973,8501.967568,5667.978378,2833.989189,2,...,0.0,0.000000,0.0,70.000000,66.000000,0,2,1,140,66
2052,1,80,6,2024-03-20 22:57:14,4.301531e+06,372.193057,1.859803,1.162377,0.697426,5,...,0.0,0.000000,0.0,146.600000,289.333333,0,5,3,733,868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38773,1,80,6,2024-03-21 01:11:04,2.682293e+06,965.591861,5.219415,4.100969,1.118446,11,...,0.0,693.304877,0.0,169.818182,240.666667,0,11,3,1868,722
38792,1,80,6,2024-03-21 01:11:01,7.749537e+06,412.669803,2.064639,1.677520,0.387120,13,...,0.0,684.865903,0.0,190.461538,240.666667,0,13,3,2476,722
41892,1,6443,6,2024-03-21 02:55:45,3.083944e+03,418944.009896,1945.560417,972.780209,972.780209,3,...,0.0,0.000000,0.0,241.000000,189.666667,0,3,3,723,569
42356,1,22,6,2024-03-21 03:11:42,2.228577e+05,744.869887,13.461504,8.974336,4.487168,2,...,0.0,0.000000,0.0,54.000000,58.000000,0,2,1,108,58


In [12]:
# Save flow data
df_flow.to_csv(save_directory + "flows.csv", index=False)

Column filtering

In [13]:
# df_flow = df_flow.get(["timestamp", "class", "flow_duration", "flow_byts_s", "flow_pkts_s", "tot_fwd_pkts", "tot_bwd_pkts", "totlen_fwd_pkts", "totlen_bwd_pkts"])
# print(df_flow.shape)
# df_flow.head()

Prepare CPU data

In [14]:
df_cpu = pd.read_csv(CPU_PATH)
df_cpu = df_cpu.rename(columns={'Time': 'timestamp'})
df_cpu = df_cpu.drop("Value 1", axis=1)
# Merge CPU values by sum
df_cpu["cpu_total"] = df_cpu.drop("timestamp", axis=1).sum(axis=1)
df_cpu.head()

,timestamp,classifier,controller,database,sniffer,cpu_total
0,2024-03-29 05:39:10,0.0286,0.00574,0.000469,0.843,0.877809
1,2024-03-29 05:39:15,0.0286,0.00574,0.000469,0.843,0.877809
2,2024-03-29 05:39:20,0.0286,0.00574,0.000469,0.843,0.877809
3,2024-03-29 05:39:25,0.0286,0.00574,0.000469,0.843,0.877809
4,2024-03-29 05:39:30,0.0142,0.00483,0.000972,0.039,0.059002


In [15]:
# Save CPU data
df_cpu.to_csv(save_directory + "cpu.csv", index=False)

Convert timestamp columns to datetime objects

In [16]:
df_cpu['timestamp'] = pd.to_datetime(df_cpu['timestamp'])
df_flow['timestamp'] = pd.to_datetime(df_flow['timestamp'])

Data mapping (method 1)

In [17]:
df_flow1 = df_flow

# Create a new column in the network dataframe to store associated cpu records
df_flow1['cpu'] = None

# Loop over each flow row
for index, flow_row in df_flow1.iterrows():
    # Find the nearest cpu record according to time
    nearest_cpu_record = df_cpu.iloc[(df_cpu['timestamp'] - flow_row['timestamp']).abs().argsort()[0]]

    # Assign the nearest cpu record to the current flow row
    df_flow1.at[index, 'cpu'] = nearest_cpu_record['cpu_total']

# Convert all timestamps to total seconds from the first timestamp
df_flow1 = df_flow1.sort_values(by='timestamp')
first_timestamp = df_flow1['timestamp'].iloc[0]
df_flow1['timestamp'] = (df_flow1['timestamp'] - first_timestamp).dt.total_seconds()

# Save flow data
df_flow1.to_csv(save_directory + "data1.csv", index=False)

# Print the updated network dataframe with associated cpu records
print(df_flow1)

       class  dst_port  protocol  timestamp  flow_duration   flow_byts_s  \
105        0     10249         6        0.0   6.842613e+01  1.870630e+06   
106        0      2381         6        1.0   7.104874e+01  1.801580e+06   
109        0     31808         6        1.0   1.272177e+06  7.766215e+02   
113        0     10259         6        6.0   2.598763e+01  4.925421e+06   
114        0       500        17        7.0   0.000000e+00  0.000000e+00   
...      ...       ...       ...        ...            ...           ...   
17202      0     10249         6    33120.0   3.647804e+01  3.508960e+06   
17203      0      2381         6    33121.0   3.004074e+01  4.260880e+06   
17204      0     10259         6    33126.0   2.765656e+01  4.628198e+06   
17236      0      6443         6    33127.0   9.124560e+07  3.112479e+01   
17208      0       500        17    33137.0   0.000000e+00  0.000000e+00   

        flow_pkts_s    fwd_pkts_s    bwd_pkts_s  tot_fwd_pkts  ...  \
105    29228.5993

Data mapping (method 2)

In [18]:
df_cpu1 = df_cpu

# Function to aggregate flows for each CPU record
def aggregate_flows(cpu_row, network_df):
    # Get the current and next CPU timestamps
    current_timestamp = cpu_row['timestamp']
    next_timestamp = df_cpu1.loc[cpu_row.name + 1, 'timestamp'] if cpu_row.name + 1 < len(df_cpu) else pd.Timestamp.max

    # Filter flows that fall within the time interval
    flows_in_interval = network_df[(network_df['timestamp'] >= current_timestamp) & (network_df['timestamp'] < next_timestamp)]

    # Aggregate flow values
    aggregated_values = flows_in_interval.drop(columns=['timestamp']).sum()

     # Add a column for the number of aggregated flows
    aggregated_values['num_flows'] = len(flows_in_interval)

    return aggregated_values

# Loop over each CPU row
for index, cpu_row in df_cpu1.iterrows():
    # Aggregate flows for the current CPU row
    aggregated_values = aggregate_flows(cpu_row, df_flow)

    # Add aggregated flow values to the CPU row
    df_cpu1.loc[index, aggregated_values.index] = aggregated_values.values

# Fill NaN values with 0
df_cpu1.fillna(0, inplace=True)

# Convert all timestamps to total seconds from the first timestamp
df_cpu1 = df_cpu1.sort_values(by='timestamp')
first_timestamp = df_cpu1['timestamp'].iloc[0]
df_cpu1['timestamp'] = (df_cpu1['timestamp'] - first_timestamp).dt.total_seconds()

# Save flow data
df_cpu1.to_csv(save_directory + "data2.csv", index=False)

# Print the updated CPU dataframe
print(df_cpu1)

      timestamp  classifier  controller  database  sniffer  cpu_total  class  \
0           0.0     0.02860     0.00574  0.000469  0.84300   0.877809    0.0   
1           5.0     0.02860     0.00574  0.000469  0.84300   0.877809    0.0   
2          10.0     0.02860     0.00574  0.000469  0.84300   0.877809    0.0   
3          15.0     0.02860     0.00574  0.000469  0.84300   0.877809    0.0   
4          20.0     0.01420     0.00483  0.000972  0.03900   0.059002    0.0   
...         ...         ...         ...       ...      ...        ...    ...   
6624    33120.0     0.00832     0.00257  0.000441  0.00489   0.016221    0.0   
6625    33125.0     0.00832     0.00257  0.000441  0.00489   0.016221    0.0   
6626    33130.0     0.00832     0.00257  0.000441  0.00489   0.016221    0.0   
6627    33135.0     0.00832     0.00257  0.000441  0.00489   0.016221    0.0   
6628    33140.0     0.01580     0.00215  0.000325  0.00901   0.027285    0.0   

      dst_port  protocol  flow_duration